<a href="https://colab.research.google.com/github/sue-bin/SCDC2020/blob/master/multi_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from patsy import dmatrices
import statsmodels.api as sm;
from statsmodels.stats.outliers_influence import variance_inflation_factor
%matplotlib inline
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [3]:
import time
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
info_df_raw = pd.read_csv("/content/drive/My Drive/데이터분석/samsung/info1.csv")
feat_df_raw = pd.read_csv("/content/drive/My Drive/데이터분석/samsung/samp_cst_feat.csv")
train_df_raw = pd.read_csv("/content/drive/My Drive/데이터분석/samsung/samp_train.csv")
dtype_df_raw = pd.read_excel("/content/drive/My Drive/데이터분석/samsung/variable_dtype.xlsx")

In [5]:
info_df = info_df_raw.copy()
feat_df = feat_df_raw.copy()
train_df = train_df_raw.copy()

In [6]:
dtype_dict = dtype_df_raw.set_index('Variable_Name')['dType'].to_dict()

In [7]:
#카테고리컬 변수타입 변환
for w in feat_df_raw.columns:
  if w in dtype_dict.keys():
    if dtype_dict[w] == "categorical":
      feat_df[w] = feat_df_raw[w].astype(str)
feat_df.dtypes.head(10)

cst_id_di      int64
VAR002       float64
VAR003       float64
VAR004       float64
VAR005       float64
VAR006       float64
VAR007        object
VAR008       float64
VAR009       float64
VAR010       float64
dtype: object

In [8]:
#카테고리컬 변수들 행 이름
obj_col = []
numerical_col = [] 
for w in dtype_dict:
  if dtype_dict[w] == "categorical":
    obj_col.append(w)
  else:
    numerical_col.append(w)

In [9]:
##categorical은 다 dummy variables
for i in obj_col:
  print(feat_df[i].value_counts())
  print()

0    8633
1    1491
Name: VAR007, dtype: int64

0    5430
1    4694
Name: VAR015, dtype: int64

0    8009
1    2115
Name: VAR018, dtype: int64

0    9926
1     198
Name: VAR026, dtype: int64

0    7606
1    2518
Name: VAR059, dtype: int64

0    7324
1    2800
Name: VAR066, dtype: int64

1    8723
0    1401
Name: VAR067, dtype: int64

0    8166
1    1958
Name: VAR070, dtype: int64

0    9314
1     810
Name: VAR071, dtype: int64

0    8769
1    1355
Name: VAR077, dtype: int64

0    9738
1     386
Name: VAR078, dtype: int64

0    9515
1     609
Name: VAR094, dtype: int64

0    9158
1     966
Name: VAR096, dtype: int64

0    8956
1    1168
Name: VAR097, dtype: int64

0    9306
1     818
Name: VAR098, dtype: int64

0    8434
1    1690
Name: VAR107, dtype: int64

0    7949
1    2175
Name: VAR111, dtype: int64

0    7452
1    2672
Name: VAR124, dtype: int64

0    7249
1    2875
Name: VAR127, dtype: int64

0    9952
1     172
Name: VAR143, dtype: int64

0    9485
1     639
Name: VAR144, dtype:

In [28]:
total_df = feat_df_raw.copy()
total_df['label'] = train_df_raw['MRC_ID_DI']

In [29]:
df = total_df.copy()

In [30]:
X = df.drop(['label'], axis = 1)
Y = df['label']
X = StandardScaler().fit_transform(X)

In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
X_pca = pca.fit_transform(X)
PCA_df = pd.DataFrame(data = X_pca, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
PCA_df = pd.concat([PCA_df, df['label']], axis = 1)
PCA_df.head()

,PC1,PC2,PC3,PC4,PC5,label
0,7.279507,-3.813250,5.084972,-5.013251,-0.228789,0
1,14.681195,4.586905,-2.685685,2.918949,-3.615027,8
2,-14.916882,-1.823236,0.040112,-1.621081,-5.169978,0
3,-3.165472,4.684853,-3.915588,-4.432158,1.878800,5
4,19.599368,7.075075,4.594998,2.728053,-0.535301,6


In [ ]:
!pip install autokeras
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc1

In [27]:
import autokeras as ak

In [40]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=32445)
clf = ak.StructuredDataClassifier(
    overwrite=False,
    max_trials=100)

clf.fit(x_train, y_train, epochs=30)

predicted_y = clf.predict(x_test)

print(clf.evaluate(x_test, y_test))


Trial 31 Complete [00h 01m 15s]
val_accuracy: 0.8149898648262024

Best val_accuracy So Far: 0.8210668563842773
Total elapsed time: 00h 28m 21s
INFO:tensorflow:Oracle triggered exit
Epoch 1/30
238/238 [==============================] - 2s 7ms/step - loss: 2.1626 - accuracy: 0.3285
Epoch 2/30
238/238 [==============================] - 2s 7ms/step - loss: 1.8297 - accuracy: 0.4969
Epoch 3/30
238/238 [==============================] - 2s 7ms/step - loss: 1.5798 - accuracy: 0.5898
Epoch 4/30
238/238 [==============================] - 2s 7ms/step - loss: 1.3762 - accuracy: 0.6643
Epoch 5/30
238/238 [==============================] - 2s 7ms/step - loss: 1.2198 - accuracy: 0.7118
Epoch 6/30
238/238 [==============================] - 2s 7ms/step - loss: 1.0846 - accuracy: 0.7485
Epoch 7/30
238/238 [==============================] - 2s 7ms/step - loss: 0.9788 - accuracy: 0.7729
Epoch 8/30
238/238 [==============================] - 2s 7ms/step - loss: 0.9087 - accuracy: 0.7832
Epoch 9/30
238/238 

In [42]:
model = clf.export_model()

In [44]:
model.

ValueError: ignored